In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import random
import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from fastmri_recon.data.oasis_sequences import Masked2DSequence
from fastmri_recon.helpers.evaluate import METRIC_FUNCS, Metrics
from fastmri_recon.helpers.nn_mri import lrelu
from fastmri_recon.helpers.reconstruction import reco_and_gt_zfilled_from_val_file, reco_and_gt_net_from_val_file
from fastmri_recon.models.cascading import cascade_net
from fastmri_recon.models.kiki_sep import full_kiki_net
from fastmri_recon.models.pdnet import pdnet
from fastmri_recon.models.unet import unet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
np.random.seed(0)

In [3]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/OASIS_data/'

# generators
AF = 4
train_gen = Masked2DSequence(
    train_path,
    af=AF,
    inner_slices=32,
    scale_factor=1e-2,
    seed=0,
    rand=True,
    val_split=0.1,
)
val_gen = train_gen.val_sequence
n_train = 1000
n_val = 2
random.seed(0)
train_gen.filenames = random.sample(train_gen.filenames, n_train)
val_gen.filenames = random.sample(val_gen.filenames, n_val)

In [5]:
all_net_params = [
#     {
#         'name': 'unet',
#         'init_function': unet,
#         'run_params': {
#             'n_layers': 4,
#             'pool': 'max',
#             "layers_n_channels": [16, 32, 64, 128],
#             'layers_n_non_lins': 2,
#             'input_size': (320, 320, 1),
#         },
#         'val_gen': val_gen_zero,
#         'run_id': 'unet_af4_1569210349',
#     },
#     {
#         'name': 'pdnet',
#         'init_function': pdnet,
#         'run_params': {
#             'n_primal': 5,
#             'n_dual': 5,
#             'n_iter': 10,
#             'n_filters': 32,
#         },
#         'val_gen': val_gen_scaled,
#         'run_id': 'pdnet_af4_1568384763',
#     },
    {
        'name': 'cascadenet',
        'init_function': cascade_net,
        'run_params': {
            'n_cascade': 5,
            'n_convs': 5,
            'n_filters': 48,
            'noiseless': True,
        },
        'run_id': 'cascadenet_af4_oasis_1569491836',
    },
#     {
#         'name': 'kikinet',
#         'init_function': kiki_net,
#         'run_params': {
#             'n_cascade': 2,
#             'n_convs': 25,
#             'n_filters': 32,
#             'noiseless': True,
#         },
#         'val_gen': val_gen_scaled,
#         'run_id': 'kikinet_af4_1568724379',
#     },
        {
        'name': 'kikinet-sep',
        'init_function': full_kiki_net,
        'run_params': {
            'n_convs': 25,
            'n_filters': 48,
            'noiseless': True,
            'activation': lrelu,
        },
        'run_id': 'kikinet_sep_I2_af4_oasis_1571452949',
        'epoch': 50,
    },
]

In [6]:
def unpack_model(init_function=None, run_params=None, run_id=None, epoch=300, **dummy_kwargs):
    model = init_function(input_size=(None, None, 1), fastmri=False, **run_params)
    chkpt_path = f'../checkpoints/{run_id}-{epoch}.hdf5'
    model.load_weights(chkpt_path)
    return model

def metrics_for_params(name=None, **net_params):
    model = unpack_model(**net_params)
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
        reco_and_gt_net_from_val_file(*val_gen[i], model)
        for i in tqdm_notebook(range(len(val_gen)), desc=f'Val files for {name}')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos)
    return metrics


def metrics_zfilled():
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
        reco_and_gt_zfilled_from_val_file(*val_gen[i], crop=False)
        for i in tqdm_notebook(range(len(val_gen)), desc='Val files for z-filled')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Stats for z-filled'):
        metrics.push(images, im_recos)
    return metrics

In [7]:
%%time
metrics = []
for net_params in all_net_params:
    metrics.append((net_params['name'], metrics_for_params(**net_params)))
    
metrics.append(('zfilled', metrics_zfilled()))

W1019 11:22:51.875104 140514566358784 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/nn_mri.py:82: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


W1019 11:23:12.160444 140514566358784 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.




CPU times: user 20.8 s, sys: 4.68 s, total: 25.5 s
Wall time: 39.5 s


In [8]:
metrics

[('cascadenet', PSNR = 32.59 +/- 7.896 SSIM = 0.9009 +/- 0.09883),
 ('kikinet-sep', PSNR = 19.45 +/- 4.042 SSIM = 0.356 +/- 0.1048),
 ('zfilled', PSNR = 26.93 +/- 4.438 SSIM = 0.6744 +/- 0.08495)]

In [9]:
metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())

In [10]:
# import pickle
# with open('metrics_net_rec_oasis', 'wb') as f:
#     pickle.dump(metrics, f)

In [11]:
def n_model_params_for_params(reco_function=None, val_gen=None, name=None, **net_params):
    model = unpack_model(**net_params)
    n_params = model.count_params()
    return n_params

In [12]:
%%time
n_params = {}
for net_params in all_net_params:
    n_params[net_params['name']] =  n_model_params_for_params(**net_params)
    
n_params['zfilled'] =  0

CPU times: user 8.12 s, sys: 94.3 ms, total: 8.22 s
Wall time: 8.1 s


In [31]:
def runtime_for_params(name=None, **net_params):
    model = unpack_model(**net_params)
    data = val_gen[0]
    start = time.time()
    reco_and_gt_net_from_val_file(*data, model)
    end = time.time()
    return end - start

def runtime_zfilled():
    data = val_gen[0]
    start = time.time()
    reco_and_gt_zfilled_from_val_file(*data, crop=False)
    end = time.time()
    return end - start

In [32]:
%%time
runtimes = {}
for net_params in tqdm_notebook(all_net_params):
    runtimes[net_params['name']] =  runtime_for_params(**net_params)
    
runtimes['zfilled'] = runtime_zfilled()


CPU times: user 19.1 s, sys: 1.08 s, total: 20.2 s
Wall time: 21.2 s


In [33]:
metrics_table = pd.DataFrame(
    index=[name for name, _ in metrics], 
    columns=['PSNR-mean (std) (dB)', 'SSIM-mean (std)', '# params', 'Runtime (s)'],
)
for name, m in metrics:
    metrics_table.loc[name, 'PSNR-mean (std) (dB)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['PSNR'].mean(), 
        std=m.metrics['PSNR'].stddev(),
    )
    metrics_table.loc[name, 'SSIM-mean (std)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['SSIM'].mean(), 
        std=m.metrics['SSIM'].stddev(),
    )
    metrics_table.loc[name, '# params'] = "{}".format(
        n_params[name], 
     )
    metrics_table.loc[name, 'Runtime (s)'] = "{runtime:.4}".format(
        runtime=runtimes[name], 
    )

In [34]:
metrics_table

,PSNR-mean (std) (dB),SSIM-mean (std),# params,Runtime (s)
kikinet-sep,19.45 (2.021),0.356 (0.0524),1999304,4.448
zfilled,26.93 (2.219),0.6744 (0.04248),0,0.156
cascadenet,32.59 (3.948),0.9009 (0.04942),424570,2.117
